# Greedy Search

## Definição dos hiper parâmetros

In [ ]:
# Importa Pandas e find_best_hyperparameter
import pandas as pd
from find_best_hyperparameters import AtividadeTres

# Carrega Atividade 3
a3 = AtividadeTres(26)

# Define hiper parâmetros para Greedy Search
hyper_params = {}

hyper_params['nb'] = {
    'model_name': 'MultinomialNB',
    'params': a3.create_param_grid({   
        # Explicar alpha
        'alpha': [0.1, 0.25, 0.5, 0.75, 1.0, 2.0, 5.0],
        # Aprender probabilidades anteriores de classe
        'fit_prior': [True, False]
    })
}

hyper_params['lr'] = {
    'model_name': 'LogisticRegression',
    'params': a3.create_param_grid({
        # Força da regularização
        'C': [0.1, 0.25, 1.0, 5.0, 10.0, 20.0],
        # Algoritmos de otimização
        'solver': ['lbfgs', 'liblinear', 'sag'],
        # Tipos de regularização
        'penalty': ['l1', 'l2', 'elasticnet', None],
        # Iterações máximas
        'max_iter': [100, 200, 500, 1000]
    })
}

hyper_params['svm'] = {
    'model_name': 'LinearSVC',
    'params': a3.create_param_grid({
        # Força da regularização
        'C': [0.01, 0.1, 1, 5, 10, 50, 100],
        # Função de perda
        'loss': ['hinge', 'squared_hinge'],
        # Iterações máximas
        'max_iter': [100, 250, 500, 1000, 5000],
        # Critério de tolerância para parar
        'tol': [1e-3, 1e-4, 1e-5]
    })
}

## Base Classic4.csv

In [ ]:
# Leitura da base Classic4.csv
BASE='classic4'
FILENAME='classic4.csv'
cstr = a3.load_data(FILENAME)
cstr.head()

In [ ]:
# Separação dos dados de treino e teste
train, test, series = a3.split_data(data=cstr, text_col='text', class_col='class', test_size=0.2)

# Salva resultados
a3.save_data_frame(train, BASE + '_train.csv')
a3.save_data_frame(test, BASE + '_test.csv')

# Mostra primeiras linhas do teste
test.head()

In [ ]:
# Greedy Search para NB, LR and SVM
best_greedy = {}
greedy = None

for item in hyper_params:
    # Testa parâmetros com Greedy Search
    greedy = pd.DataFrame(a3.greedy_search(
        model_name=hyper_params[item]['model_name'],
        param_grid=hyper_params[item]['params'],
        series=series
    ))
    # Salva resultados em arquivo
    a3.save_data_frame(greedy, BASE + '_greedy_' + item + '.csv')
    # Resgata melhor resultado para treinamento
    max_score = greedy['score'].max()
    max_rows = greedy[greedy['score'] == max_score]
    best_greedy[item] = max_rows.sample(1).reset_index(drop=True).iloc[0].to_dict()
    
# Exibe valores no notebook
pd.DataFrame(best_greedy)

In [ ]:
# Treina modelo com melhores parâmetros
metrics = {}

for item in best_greedy:
    # Remove key score
    try:
        best_greedy[item].pop('score')
    except:
        pass
    metrics[item] = a3.train(model_name=item, params=best_greedy[item], series=series, base=BASE)

a3.save_data_frame(pd.DataFrame(metrics), BASE + '_metrics.csv')

# Exibe valores no notebook
pd.DataFrame(metrics)

## Base Dmoz-Sports.csv

In [ ]:
# Leitura da base Dmoz-Sports.csv
BASE='Dmoz-Sports'
FILENAME='Dmoz-Sports.csv'
cstr = a3.load_data(FILENAME)
cstr.head()

In [ ]:
# Separação dos dados de treino e teste
train, test, series = a3.split_data(data=cstr, text_col='text', class_col='class', test_size=0.2)

# Salva resultados
a3.save_data_frame(train, BASE + '_train.csv')
a3.save_data_frame(test, BASE + '_test.csv')

# Mostra primeiras linhas do teste
test.head()

In [ ]:
# Greedy Search para NB, LR and SVM
best_greedy = {}
greedy = None

for item in hyper_params:
    # Testa parâmetros com Greedy Search
    greedy = pd.DataFrame(a3.greedy_search(
        model_name=hyper_params[item]['model_name'],
        param_grid=hyper_params[item]['params'],
        series=series
    ))
    # Salva resultados em arquivo
    a3.save_data_frame(greedy, BASE + '_greedy_' + item + '.csv')
    # Resgata melhor resultado para treinamento
    max_score = greedy['score'].max()
    max_rows = greedy[greedy['score'] == max_score]
    best_greedy[item] = max_rows.sample(1).reset_index(drop=True).iloc[0].to_dict()
    
# Exibe valores no notebook
pd.DataFrame(best_greedy)

In [ ]:
# Treina modelo com melhores parâmetros
metrics = {}

for item in best_greedy:
    # Remove key score
    try:
        best_greedy[item].pop('score')
    except:
        pass
    metrics[item] = a3.train(model_name=item, params=best_greedy[item], series=series, base=BASE)

a3.save_data_frame(pd.DataFrame(metrics), BASE + '_metrics.csv')

# Exibe valores no notebook
pd.DataFrame(metrics)